### 1. 沿着曲线插入文字

沿着图中彩虹的曲线边缘插入用中文写的一句话

**预处理**：导入 *PIL* 库，以便在图像中显示中文

In [143]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
# 读取彩虹图像
image = cv2.imread('rainbow_1.jpg')

对图像滤波以增强图像的细节，便于检测到彩虹

In [144]:
detailEnhance_image = cv2.detailEnhance(image, sigma_s=100, sigma_r=0.1)

由于彩虹中有**红色**，而图中其他元素没有，所以将图像转换为 *HSV* 颜色空间并利用颜色范围创建掩膜

In [145]:
hsv_image = cv2.cvtColor(detailEnhance_image, cv2.COLOR_BGR2HSV)
# 设置彩虹的 HSV 颜色范围
# 红色范围
lower_rainbow = np.array([0, 65, 65])
upper_rainbow = np.array([10, 200, 200])
mask = cv2.inRange(hsv_image, lower_rainbow, upper_rainbow)

对图像应用 *canny* 边缘检测， 将边缘检测结果和上面的掩膜进行“与”操作，得到彩虹边缘。

In [146]:
# 将图像转换为灰度图像
gray_image = cv2.cvtColor(detailEnhance_image, cv2.COLOR_BGR2GRAY)

# 应用 Canny 边缘检测
edges = cv2.Canny(gray_image, threshold1=50, threshold2=80)

# 与掩膜进行逻辑与操作，提取彩虹的边缘
edges_filtered = cv2.bitwise_and(edges, edges, mask=mask)

将图像转换为 *PIL* 图像，并定义中文文字和字体。

In [147]:
# 将图像转换为 PIL 图像
image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
draw = ImageDraw.Draw(image_pil)

# 定义中文文字和字体
text = '这是一条彩虹'              # 要显示的中文文字
font_path = 'simhei.ttf'  # 字体文件路径
font_size = 40            # 字体大小
font = ImageFont.truetype(font_path, font_size)
font_color = (0, 255, 0)  # 字体颜色

**提取边缘坐标点**

In [148]:
edge_points = np.column_stack(np.where(edges_filtered > 0))
sorted_edge_points = edge_points[edge_points[:, 1].argsort()]

In [149]:
# 在边缘上插入中文文字
i = 0
for point in sorted_edge_points[::30]:  # 取样点减少文字数量
    y, x = point
    if i < len(text):  # 确保不会超出文本长度
        draw.text((x, y), text[i], font=font, fill=font_color)
        i += 1  # 递增 i，指向下一个字符
    else:
       i = 0  # 当所有文本绘制完成后跳出循环


In [150]:
# 转换回 OpenCV 图像
image_with_text = np.array(image_pil)
image_with_text = cv2.cvtColor(image_with_text, cv2.COLOR_RGB2BGR)

In [151]:
# 显示结果
cv2.imshow('edge', edges_filtered)
cv2.imshow('Image with Text', image_with_text)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 2.图像目标数数
写程序自动算出图中鸟的个数

In [16]:
import cv2
import numpy as np

image = cv2.imread('birds.jpeg')

**双边滤波去噪**

In [17]:
filtered_image = cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)

**将图像转换为 *HSV* 颜色空间**

In [18]:
hsv_image = cv2.cvtColor(filtered_image, cv2.COLOR_BGR2HSV)

**设置鸟的颜色范围并创建掩膜**

In [19]:
# 设置鸟的黑色范围
lower_black = np.array([0, 0, 0])
upper_black = np.array([180, 255, 255])

# 创建掩膜
mask_black = cv2.inRange(hsv_image, lower_black, upper_black)

# 设置鸟的黄色范围
lower_yellow = np.array([26, 43, 46])
upper_yellow = np.array([34, 255, 255])

# 创建掩膜
mask_yellow = cv2.inRange(hsv_image, lower_yellow, upper_yellow)

# 合并掩膜
mask = cv2.bitwise_or(mask_black, mask_yellow)

**使用 Canny 算子进行边缘检测，并与掩膜进行逻辑与操作，提取感兴趣区域的边缘**

In [20]:
# 使用 Canny 算子进行边缘检测
gray = cv2.cvtColor(filtered_image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150)

# 与掩膜进行逻辑与操作，提取感兴趣区域的边缘
edges_filtered = cv2.bitwise_and(edges, edges, mask=mask)

**查找轮廓，设置轮廓面积阈值**

In [21]:
# 查找轮廓
contours, _ = cv2.findContours(edges_filtered, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

bird_count = 0
min_area_threshold = 5  # 最小面积阈值，根据实际图片调整
max_area_threshold = 80  # 最大面积阈值，避免过大物体被识别为鸟

**判断检测对象是否是鸟**

In [22]:
for contour in contours:
    area = cv2.contourArea(contour)

    # 根据面积判断是否可能是鸟类
    if min_area_threshold < area < max_area_threshold:
        x, y, w, h = cv2.boundingRect(contour)
        # 计算长宽比 (Aspect Ratio)
        aspect_ratio = float(w) / h
        if 0.5 < aspect_ratio < 2.0:  # 假设鸟的长宽比在 0.5 到 2.0 之间
            # 画出识别到的鸟的边框
            cv2.rectangle(image, (x, y), (x + w, y + h), (36, 255, 12), 1)
            bird_count += 1

**显示结果**

In [24]:
print(f"检测到 {bird_count} 只鸟。")
cv2.imshow('Birds Detected', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

检测到 68 只鸟。
